# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770882812452                   -0.53    8.0    174ms
  2   -2.772141348290       -2.90       -1.31    1.0    123ms
  3   -2.772170013210       -4.54       -2.57    1.0    111ms
  4   -2.772170711354       -6.16       -3.62    2.0    129ms
  5   -2.772170722517       -7.95       -4.24    1.0    119ms
  6   -2.772170722969       -9.35       -4.91    1.0    122ms
  7   -2.772170723015      -10.34       -5.63    2.0    136ms
  8   -2.772170723015      -12.49       -6.42    1.0    124ms
  9   -2.772170723015      -13.76       -7.04    1.0    122ms
 10   -2.772170723015   +    -Inf       -7.89    2.0    139ms
 11   -2.772170723015      -13.88       -8.44    1.0    586ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770751306822                   -0.52    9.0    190ms
  2   -2.7

1.7735580388181054

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770756214246                   -0.52    9.0    181ms
  2   -2.772059806606       -2.88       -1.32    1.0    158ms
  3   -2.772082981762       -4.63       -2.43    1.0    510ms
  4   -2.772083333716       -6.45       -3.12    1.0    118ms
  5   -2.772083417446       -7.08       -4.18    2.0    127ms
  6   -2.772083417700       -9.60       -4.54    1.0    114ms
  7   -2.772083417809       -9.96       -5.87    1.0    116ms
  8   -2.772083417811      -11.76       -6.74    2.0    131ms
  9   -2.772083417811      -14.27       -6.87    2.0    132ms
 10   -2.772083417811   +  -14.65       -7.85    1.0    125ms
 11   -2.772083417811      -15.05       -8.37    1.0    123ms
Solving response problem
[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; st